### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Substance_Use_Disorder/GSE222468'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Check if gene expression data is available
is_gene_available = False  # Based on series title, this is a methylation dataset

# 2. Variable Availability and Data Type Conversion
# Check for the 'Substance_Use_Disorder' variable
# The trait is related to psychiatric disorders; thus, we might use the 'burden_of_psychiatric_disease' as a proxy.
for key, values in sample_characteristics.items():
    if any('burden_of_psychiatric_disease' in value for value in values):
        trait_row = key
        break

# Check for the 'age' variable
for key, values in sample_characteristics.items():
    if any('age:' in value for value in values):
        age_row = key
        break

# Check for the 'gender' variable
for key, values in sample_characteristics.items():
    if any('Sex:' in value for value in values):
        gender_row = key
        break

# Data type conversion functions
def convert_trait(value):
    if 'burden_of_psychiatric_disease' in value:
        val = value.split(':')[1].strip()
        try:
            return int(val)  # assuming burden is a categorical variable
        except ValueError:
            return None
    return None

def convert_age(value):
    if 'age:' in value:
        val = value.split(':')[1].strip()
        try:
            return float(val)
        except ValueError:
            return None
    return None

def convert_gender(value):
    if 'Sex:' in value:
        val = value.split(':')[1].strip().lower()
        if val == 'female':
            return 0
        elif val == 'male':
            return 1
    return None

# Save cohort information
save_cohort_info('GSE222468', './preprocessed/Substance_Use_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'burden_of_psychiatric_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Substance_Use_Disorder/trait_data/GSE222468.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
